In [3]:
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
from plotly.subplots import make_subplots
import plotly.graph_objects as go

plt.rcParams["figure.figsize"] = (20, 9)


In [7]:
# Blink detection, duration and perclose index using exponetial moving average.
# this looks good for all kind of case but facing problem while calculating blink duration and same is reflacting in perclose index.
path = f"./Ashish_data/"
csvfiles = []
number_of_blinks = []
for x in os.listdir(path):
    if x.endswith(".csv"):
        csvfiles.append(x)
    # print(videofilename)
csvfiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
blink_duration = pd.DataFrame(columns=["Start_blink", "End_blink", "Blink_frame", "No_of_frames"])
perclos_value = list()
for x in csvfiles:
    df_alert = pd.read_csv(f"./Ashish_data/{x}")
    # remove rows where facemesh could not detect the face
    df_alert = df_alert.loc[(df_alert["EAR"] != 0)]
    df_alert = df_alert.reset_index()  # reindexing to identify the blink duration
    # after reset_index(), one additional column is added which does not required
    df_alert.drop(columns=["index"], axis=1, inplace=True)
    df_alert["NewEMA"] = df_alert["NewEAR"].ewm(4).mean()  # exponetial weighted average moving a
    startframe = list()
    endframe = list()
    frame_no = []
    print(x)
    for i in range(df_alert.shape[0]):
        if df_alert["NewEMA"].iloc[i] < df_alert["NewEMA"].iloc[i-4]*0.85 and i > 4 and (df_alert["FN"].iloc[i] - df_alert["FN"].iloc[i-1]) < 2 and df_alert["NewEAR"].iloc[i]<0.14 :
            if len(frame_no) == 0 or (df_alert["FN"].iloc[i] - frame_no[-1] > 8):
                frame_no.append(df_alert["FN"].iloc[i])
            else:
                if (df_alert["NewEMA"].iloc[i] < df_alert["NewEMA"].iloc[i-1]):
                    frame_no[-1] = df_alert["FN"].iloc[i]
    number_of_blinks.append(len(frame_no))
    # print(frame_no)
    for i in range(len(frame_no)):
        start_frame, end_frame, blinked_frame = 0, 0, 0
        blinked_frame = frame_no[i]
        earemv = df_alert["NewEMA"].loc[df_alert["FN"]== blinked_frame].iloc[0]

        if i == 0:
            sf = 0
        else:
            sf = df_alert.index[df_alert["FN"] == blink_duration["End_blink"].iloc[i-1]][0]
        # print("end_blink",blink_duration["End_blink"].iloc[i-1])

        if i == len(frame_no)-1:
            en = df_alert.shape[0]
        else:
            en = df_alert.index[df_alert["FN"] == frame_no[i+1]][0]
        # print(sf)
        # print("start {} and end frame {}".format(sf,en))
        
        for j in range(df_alert.index[df_alert["FN"] == frame_no[i]][0], sf, -1):
        # print(j)
            if (df_alert["NewEMA"].iloc[j] < (earemv/0.92)) and df_alert["NewEAR"].iloc[j] < 0.1:
                start_frame = df_alert["FN"].iloc[j]
                continue
            else:
                start_frame = df_alert["FN"].iloc[j]
            # print("start frame",start_frame)
                break
        # print("threshold",(earemv/0.92))
        # print("------------")
        for j in range(df_alert.index[df_alert["FN"] == frame_no[i]][0], en):
        # print(df_alert["NewEMA"].iloc[j])
            if ((df_alert["NewEMA"].iloc[j]) > (earemv/0.92)) and (df_alert["NewEAR"].iloc[j] > 0.1):
                end_frame = df_alert["FN"].iloc[j]
                break
            else:
                end_frame = df_alert["FN"].iloc[j]
                continue
            
        # print(start_frame)
        blink_duration.loc[blink_duration.shape[0]] = [start_frame, end_frame, blinked_frame, (end_frame-start_frame)]
    file = blink_duration.to_csv(x.split("_")[0]+"_Blink_Duration.csv")
    frame_no.clear()
    blink_duration = blink_duration.drop(blink_duration.index[0:])


print(number_of_blinks)
# file = df_alert.to_csv("Test.csv")


1_EAR.csv
2_EAR.csv
3_EAR.csv
4_EAR.csv
5_EAR.csv
6_EAR.csv
7_EAR.csv
8_EAR.csv
9_EAR.csv
10_EAR.csv
11_EAR.csv
12_EAR.csv
13_EAR.csv
14_EAR.csv
15_EAR.csv
16_EAR.csv
17_EAR.csv
18_EAR.csv
19_EAR.csv
20_EAR.csv
21_EAR.csv
22_EAR.csv
23_EAR.csv
24_EAR.csv
25_EAR.csv
26_EAR.csv
[168, 139, 985, 798, 1332, 947, 456, 210, 68, 97, 357, 1126, 1528, 212, 523, 671, 26, 191, 137, 308, 540, 843, 601, 71, 51, 1747]


In [7]:
# count nomber of frames where face is not detected
path = f"./Ashish_data/Samsung/"
csvfiles = []
count = []
for x in os.listdir(path):
    if x.endswith(".csv"):
        csvfiles.append(x)
    # print(videofilename)
csvfiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
for x in csvfiles:
    df_alert = pd.read_csv(f"./Ashish_data/Samsung/{x}")
    count.append((df_alert["NewEAR"] == 0).sum())
print(count)


[145, 0, 185, 2, 21337, 0, 155, 43, 27, 350, 25]


In [ ]:
pd.set_option('display.max_rows', None)
df_alert

In [8]:
path = f"./Ashish_data/Samsung/"
csvfiles = []
number_of_blinks = []
for x in os.listdir(path):
    if x.endswith(".csv"):
        csvfiles.append(x)
    # print(videofilename)
csvfiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
blink_duration = pd.DataFrame(columns=["Start_blink", "End_blink", "Blink_frame", "No_of_frames"])
perclos_value = list()
for x in csvfiles:
    df_alert = pd.read_csv(f"./Ashish_data/Samsung/{x}")
    # remove rows where facemesh could not detect the face
    df_alert = df_alert.loc[(df_alert["EAR"] != 0)]
    df_alert = df_alert.reset_index()  # reindexing to identify the blink duration
    # after reset_index(), one additional column is added which does not required
    df_alert.drop(columns=["index"], axis=1, inplace=True)
    df_alert["NewEMA"] = df_alert["NewEAR"].ewm(4).mean()  # exponetial weighted average moving a
    startframe = list()
    endframe = list()
    frame_no = []
    print(x)
    for i in range(df_alert.shape[0]):
        if df_alert["NewEMA"].iloc[i] < df_alert["NewEMA"].iloc[i-4]*0.85 and i > 4 and (df_alert["FN"].iloc[i] - df_alert["FN"].iloc[i-1]) < 2 and df_alert["EAR"].iloc[i]<0.14:
            if len(frame_no) == 0 or (df_alert["FN"].iloc[i] - frame_no[-1] > 8):
                frame_no.append(df_alert["FN"].iloc[i])
            else:
                if (df_alert["NewEMA"].iloc[i] < df_alert["NewEMA"].iloc[i-1]):
                    frame_no[-1] = df_alert["FN"].iloc[i]
    number_of_blinks.append(len(frame_no))
    # print(frame_no)
    for i in range(len(frame_no)):
        start_frame, end_frame, blinked_frame = 0, 0, 0
        blinked_frame = frame_no[i]
        earemv = df_alert["NewEMA"].loc[df_alert["FN"]== blinked_frame].iloc[0]

        if i == 0:
            sf = 0
        else:
            sf = df_alert.index[df_alert["FN"] == blink_duration["End_blink"].iloc[i-1]][0]
        # print("end_blink",blink_duration["End_blink"].iloc[i-1])

        if i == len(frame_no)-1:
            en = df_alert.shape[0]
        else:
            en = df_alert.index[df_alert["FN"] == frame_no[i+1]][0]
        # print(sf)
        # print("start {} and end frame {}".format(sf,en))
        
        for j in range(df_alert.index[df_alert["FN"] == frame_no[i]][0], sf, -1):
        # print(j)
            if (df_alert["NewEMA"].iloc[j] < (earemv/0.92)) and df_alert["NewEAR"].iloc[j] < 0.1:
                start_frame = df_alert["FN"].iloc[j]
                continue
            else:
                start_frame = df_alert["FN"].iloc[j]
            # print("start frame",start_frame)
                break
        # print("threshold",(earemv/0.92))
        # print("------------")
        for j in range(df_alert.index[df_alert["FN"] == frame_no[i]][0], en):
        # print(df_alert["NewEMA"].iloc[j])
            if ((df_alert["NewEMA"].iloc[j]) > (earemv/0.92)) and (df_alert["NewEAR"].iloc[j] > 0.1):
                end_frame = df_alert["FN"].iloc[j]
                break
            else:
                end_frame = df_alert["FN"].iloc[j]
                continue
            
        # print(start_frame)
        blink_duration.loc[blink_duration.shape[0]] = [start_frame, end_frame, blinked_frame, (end_frame-start_frame)]
    file = blink_duration.to_csv(x.split("_")[0]+"_Blink_Duration.csv")
    frame_no.clear()
    blink_duration = blink_duration.drop(blink_duration.index[0:])


print(number_of_blinks)
# file = df_alert.to_csv("Test.csv")


1_EAR.csv
2_EAR.csv
3_EAR.csv
4_EAR.csv
5_EAR.csv
6_EAR.csv
7_EAR.csv
8_EAR.csv
9_EAR.csv
10_EAR.csv
11_EAR.csv
[73, 30, 313, 38, 282, 4, 192, 326, 51, 662, 91]


In [36]:
#Blink detection, blink duration and perclose index using EAr value
# Problem with this is what if EAR wont reach to less than 0.1 but taking care of those things by taking hyper parameter that is 0.68 that means EAr values goes down to 68% in 
# the time span of 4 frame window.
path = f"./Ashish_data/"
csvfiles = []
number_of_blinks = []
for x in os.listdir(path):
    if x.endswith(".csv"):
        csvfiles.append(x)
    # print(videofilename)
csvfiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
blink_duration = pd.DataFrame(columns=["Start_blink", "End_blink", "Blink_frame", "No_of_frames"])
perclos_value = list()
for x in csvfiles:
    df_alert = pd.read_csv(f"./Ashish_data/{x}")
    # remove rows where facemesh could not detect the face
    df_alert = df_alert.loc[(df_alert["EAR"] != 0)]
    df_alert = df_alert.reset_index()  # reindexing to identify the blink duration
    # after reset_index(), one additional column is added which does not required
    df_alert.drop(columns=["index"], axis=1, inplace=True)
    df_alert["NewEMA"] = df_alert["NewEAR"].ewm(4).mean()  # exponetial weighted average moving a
    startframe = list()
    endframe = list()
    frame_no = []
    print(x)
    for i in range(df_alert.shape[0]):
        if df_alert["NewEAR"].iloc[i] < df_alert["NewEAR"].iloc[i-4]*0.68 and i > 4 and (df_alert["FN"].iloc[i] - df_alert["FN"].iloc[i-1]) < 2 and df_alert["NewEAR"].iloc[i]<0.11:
            if len(frame_no) == 0 or (df_alert["FN"].iloc[i] - frame_no[-1] > 8):
                frame_no.append(df_alert["FN"].iloc[i])
            else:
                if (df_alert["NewEAR"].iloc[i] < df_alert["NewEAR"].iloc[i-1]):
                    frame_no[-1] = df_alert["FN"].iloc[i]
    number_of_blinks.append(len(frame_no))
    # print(frame_no)
    for i in range(len(frame_no)):
        start_frame, end_frame, blinked_frame = 0, 0, 0
        blinked_frame = frame_no[i]
        earemv = df_alert["NewEAR"].loc[df_alert["FN"]== blinked_frame].iloc[0]

        if i == 0:
            sf = 0
        else:
            sf = df_alert.index[df_alert["FN"] == blink_duration["End_blink"].iloc[i-1]][0]
        # print("end_blink",blink_duration["End_blink"].iloc[i-1])

        if i == len(frame_no)-1:
            en = df_alert.shape[0]
        else:
            en = df_alert.index[df_alert["FN"] == frame_no[i+1]][0]
        # print(sf)
        # print("start {} and end frame {}".format(sf,en))
        
        for j in range(df_alert.index[df_alert["FN"] == frame_no[i]][0], sf, -1):
        # print(j)
            if (df_alert["NewEAR"].iloc[j] < (earemv/0.84)) and df_alert["NewEAR"].iloc[j] < 0.1:
                start_frame = df_alert["FN"].iloc[j]
                continue
            else:
                start_frame = df_alert["FN"].iloc[j]
            # print("start frame",start_frame)
                break
        # print("threshold",(earemv/0.92))
        # print("------------")
        for j in range(df_alert.index[df_alert["FN"] == frame_no[i]][0], en):
        # print(df_alert["NewEMA"].iloc[j])
            if ((df_alert["NewEAR"].iloc[j]) > (earemv/0.84)) and (df_alert["NewEAR"].iloc[j] > 0.1):
                end_frame = df_alert["FN"].iloc[j]
                break
            else:
                end_frame = df_alert["FN"].iloc[j]
                continue
            
        # print(start_frame)
        blink_duration.loc[blink_duration.shape[0]] = [start_frame, end_frame, blinked_frame, (end_frame-start_frame)]
    file = blink_duration.to_csv(x.split("_")[0]+"_NewEAR_Blink_Duration.csv")
    frame_no.clear()
    blink_duration = blink_duration.drop(blink_duration.index[0:])


print(number_of_blinks)
# file = df_alert.to_csv("Test.csv")


1_EAR.csv
2_EAR.csv
3_EAR.csv
4_EAR.csv
5_EAR.csv
6_EAR.csv
7_EAR.csv
8_EAR.csv
9_EAR.csv
10_EAR.csv
11_EAR.csv
12_EAR.csv
13_EAR.csv
14_EAR.csv
15_EAR.csv
16_EAR.csv
17_EAR.csv
18_EAR.csv
19_EAR.csv
20_EAR.csv
21_EAR.csv
22_EAR.csv
23_EAR.csv
24_EAR.csv
25_EAR.csv
26_EAR.csv
[157, 154, 980, 828, 1382, 956, 480, 239, 83, 101, 416, 1199, 1574, 224, 576, 668, 20, 209, 137, 328, 548, 819, 623, 70, 57, 1652]


In [ ]:
[168, 139, 985, 798, 1332, 947, 456, 210, 68, 97, 357, 1126, 1528, 212, 523, 671, 26, 191, 137, 308, 540, 843, 601, 71, 51, 1747]

[153, 154, 978, 820, 1372, 952, 480, 239, 83, 100, 411, 1189, 1561, 223, 575, 663, 20, 208, 131, 328, 541, 817, 620, 68, 56, 1648]


In [29]:
dataframe= pd.read_csv("Day1_driver1.csv")
class_lable = []
dataframe = dataframe.loc[(dataframe["EAR"] != 0)]
dataframe = dataframe.reset_index()  # reindexing to identify the blink duration
# after reset_index(), one additional column is added which does not required
dataframe.drop(columns=["index"], axis=1, inplace=True)
dataframe.drop(dataframe.columns[[1,2]],axis=1,inplace=True)
dataframe.rename(columns={"level_0":"FN"}, inplace=True)
dataframe["NewEMA"] = dataframe["NewEAR"].ewm(4).mean()  # exponetial weighted average moving a
for i in range(dataframe.shape[0]):
    if dataframe["EAR_L"].iloc[i]<0.1 or dataframe["EAR_R"].iloc[i]<0.1:
        class_lable.append(1)
    else:
        class_lable.append(0)
dataframe["class_lable"] = class_lable

In [30]:
dataframe.to_csv("drive_data_with_lable.csv")


In [31]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
entire_data = dataframe[["NewEAR","NewEMA","class_lable"]]


In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(entire_data.iloc[:,:-1],entire_data.iloc[:,-1],random_state=1,shuffle=False)
clr = MLPClassifier(max_iter=250,verbose=True,hidden_layer_sizes = (5,2,),tol=0.00001).fit(X_train,Y_train)
# clr = SVC(kernel='poly',degree=3,max_iter=1000).fit(X_train,Y_train)


Iteration 1, loss = 0.58527803
Iteration 2, loss = 0.38125427
Iteration 3, loss = 0.32700358
Iteration 4, loss = 0.26609043
Iteration 5, loss = 0.16387729
Iteration 6, loss = 0.12696266
Iteration 7, loss = 0.11309239
Iteration 8, loss = 0.10672605
Iteration 9, loss = 0.10357239
Iteration 10, loss = 0.10187373
Iteration 11, loss = 0.10091873
Iteration 12, loss = 0.10024965
Iteration 13, loss = 0.09989380
Iteration 14, loss = 0.09953617
Iteration 15, loss = 0.09942814
Iteration 16, loss = 0.09925077
Iteration 17, loss = 0.09922117
Iteration 18, loss = 0.09913310
Iteration 19, loss = 0.09904408
Iteration 20, loss = 0.09902424
Iteration 21, loss = 0.09896793
Iteration 22, loss = 0.09910168
Iteration 23, loss = 0.09903987
Iteration 24, loss = 0.09898099
Iteration 25, loss = 0.09900054
Iteration 26, loss = 0.09902346
Iteration 27, loss = 0.09893113
Iteration 28, loss = 0.09893048
Iteration 29, loss = 0.09899721
Iteration 30, loss = 0.09892349
Iteration 31, loss = 0.09900189
Iteration 32, los

In [59]:
print(len(clr.predict(X_test)))
output_list = clr.predict(X_test)
print(clr.score(X_test, Y_test))

70173
0.9595713451042424


In [34]:
dataframe= pd.read_csv("Day1_driver1.csv")
dataframe = dataframe.loc[(dataframe["EAR"] != 0)]
dataframe = dataframe.reset_index()  # reindexing to identify the blink duration
# after reset_index(), one additional column is added which does not required
dataframe.drop(columns=["index"], axis=1, inplace=True)
dataframe.drop(dataframe.columns[[1,2]],axis=1,inplace=True)
dataframe.rename(columns={"level_0":"FN"}, inplace=True)
dataframe["NewEMA"] = dataframe["NewEAR"].ewm(4).mean()  # exponetial weighted average moving a
entire_data = dataframe[["NewEAR","NewEMA"]]
output = clr.predict(entire_data)
# print(clr.score(X_test, Y_test))


In [35]:
def consec_b(sequ):
    seq_flag = False
    seq_count = 0
    for i in range (len(sequ)-1):
        if (sequ[i] == sequ[i+1] == 1) :
            if not seq_flag: seq_count += 1
            seq_flag = True
        else: seq_flag = False
    return seq_count
consec_b(output)

5638